In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Input, InputLayer, Dropout, Dense, Flatten, Embedding, Add, Concatenate
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils import class_weight

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop

import matplotlib.pyplot as plt

from sklearn.metrics import precision_recall_fscore_support

## Own code 
import import_ipynb
import metadata_options
import models_nn

importing Jupyter notebook from metadata_options.ipynb
importing Jupyter notebook from models_nn.ipynb


In [2]:
# Data with Throughput & WorkTime
df = pd.read_csv('./data/kokil dec 6 reprepare/conf_pc_worker_sem.csv')
df = df.dropna()

In [3]:
######################################
## Weighted Onehot Encoding options ##
######################################

##############
# Throughput #
##############
# TP1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# TP2: weighted by 1 linear variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# TP3 + k: weighted by 1 inverted k-power U-shaped variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# TP4 + k: weighted by 1 upright k-power U-shaped variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# (For TP3 & TP4, k=1 results in V-shaped variance, and as k>1 increases, sides will curve into U-shaped variance)

############
# Worktime #
############
# WT1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# WT2: weighted by 1 linear variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)

################
# PC agreement #
################
# PC1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# PC2: weighted by 1 linear variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# PC3: weighted by 1 PC agreement weight per annotation in each OHE, i.e. (a, b, c, d) -> (w1*a, w2*b, w3*c, w4*d)

#####################
# Input text length #
#####################
# TL1: weighted by 1 normalised number of characters per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# TL2: weighted by 1 normalised number of words per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)

###################
# Special Options #
###################
# SP1: weighted by average of TP1 and TP2 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# SP2: weighted by average of WT1 and WT2 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# SP3: weighted by average of PC1 and PC2 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# RAND_UNI: weighted by 1 uniformly distributed random number between 0 to 1 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# RAND_NORM: weighted by 1 normally distributed random number between 0 to 1 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)

# Select 1 option from each of the few variants above, e.g. TP2, WT1, PC3, TL1, SP3, and input into function
# set_OHE_pipeline_options. If not selecting TP3 or TP4, input k (option_k) will be ignored. After
# editing the options, run the entire notebook for results accordingly.

# Edit option choices here
throughput_option = 'TP1'
worktime_option = 'WT1'
pc_agreement_option = 'PC1'
textlength_option = 'TL1'
special_option = 'SP1'
k_option_for_tp = 3

df_throughput, df_worktime, df_agreement, df_textlength, df_special = metadata_options.set_OHE_pipeline_options(df, throughput_option, worktime_option, pc_agreement_option, textlength_option, special_option, k_option_for_tp)

TP1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
WT1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
PC1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
TL1: weighted by 1 normalised number of characters per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
SP1: weighted by average of TP1 and TP2 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)


In [4]:
######################################
## Model Options ##
######################################
# options: lstm, cnn, lstm-attn

model_name = 'lstm'
models_nn.MODEL_NAME = model_name

In [5]:
df.head()

,Input.sentence_id,HITId,Input.convo_id,Input.train_test_val,Input.msg_id,Input.timestamp,Input.full_text,Input.speaker,Input.reply_to,Input.speaker_intention,...,prt,punct,purpcl,quantmod,rcmod,rel,root,tmod,xcomp,xsubj
5,11,3MG8450X2OASXZ0WO9O5AH70GU3UPA,Game1-italy-germany,Train,Game1-italy-germany-3,87,It seems like there are a lot of ways that cou...,germany-Game1,Game1-italy-germany-2,Truth,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
6,12,38G0E1M85M552JXSALX4G9WI2I6UVX,Game1-italy-germany,Train,Game1-italy-germany-7,117,"Sorry Italy I've been away doing, um, German t...",germany-Game1,Game1-italy-germany-6,Truth,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,14,3HYV4299H0WQ2B4TCS7PKDQ75WHE81,Game1-italy-germany,Train,Game1-italy-germany-8,119,"I don't think I'm ready to go for that idea, h...",germany-Game1,Game1-italy-germany-7,Truth,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
8,15,3XU9MCX6VOC4P079IHIO9TCNYLGR2P,Game1-italy-germany,Train,Game1-italy-germany-9,121,I am pretty conflicted about whether to guess ...,italy-Game1,Game1-italy-germany-8,Truth,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,16,3FVBZG9CLJEK4WQS7P2GC1H2EEQH0Q,Game1-italy-germany,Train,Game1-italy-germany-9,121,I am going to take it literally and say even ...,italy-Game1,Game1-italy-germany-8,Truth,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0


In [6]:
# # Old train_test_split code
# train, test, indices_train, indices_test = train_test_split(df, indices, test_size=0.2)

# New train_test_split using Stratified Shaffled Splits
y = df["Input.deception_quadrant"].copy()
X = df.drop(["Input.deception_quadrant"], axis=1)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
splits_generator = sss.split(X, y)

for train_idx, test_idx in splits_generator:
    indices_train = train_idx
    indices_test = test_idx

train = df.take(indices_train)
test = df.take(indices_test)

In [7]:
print(df.shape, train.shape, test.shape)

(11366, 862) (9092, 862) (2274, 862)


In [8]:
# check scikit-learn version
import sklearn
print('sklearn: %s' % sklearn.__version__)

sklearn: 0.23.2


In [9]:
new_deception_train = train["Input.deception_quadrant"].copy()
new_deception_train['Input.deception_quadrant'] = train["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y_train_deception = new_deception_train['Input.deception_quadrant'].to_numpy()
deception_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_deception),
                                                 y_train_deception)
deception_class_weight_dict = dict(enumerate(deception_class_weights))

y_train_rapport = train['Answer.3rapport.yes_label'].tolist()
rapport_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_rapport),
                                                 y_train_rapport)
rapport_class_weight_dict = dict(enumerate(rapport_class_weights))

y_train_share_information = train['Answer.4shareinformation.yes_label'].tolist()
share_info_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_share_information),
                                                 y_train_share_information)
share_info_class_weight_dict = dict(enumerate(share_info_class_weights))

y_train_reasoning = train['Answer.2reasoning.yes_label'].tolist()
reasoning_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_reasoning),
                                                 y_train_reasoning)
reasoning_class_weight_dict = dict(enumerate(reasoning_class_weights))

y_train_gamemove = train['Answer.1gamemove.yes_label'].tolist()
gamemove_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_gamemove),
                                                 y_train_gamemove)
gamemove_class_weight_dict = dict(enumerate(gamemove_class_weights))

/home/kaiyuan/anaconda3/envs/TF2/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0 1], y=[1 1 1 ... 1 1 1] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/home/kaiyuan/anaconda3/envs/TF2/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0. 1.], y=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 

In [10]:
print("Class weights")
print("Deception: {} \nRapport: {} \nShare Information: {} \nReasoning: {} \nGamemove: {}".format(deception_class_weight_dict,
                                                                                                  rapport_class_weight_dict,
                                                                                                  share_info_class_weight_dict,
                                                                                                  reasoning_class_weight_dict,
                                                                                                  gamemove_class_weight_dict))

Class weights
Deception: {0: 10.192825112107624, 1: 0.525792273883877} 
Rapport: {0: 3.582348305752561, 1: 0.5811069922024799} 
Share Information: {0: 3.117969821673525, 1: 0.59549384333246} 
Reasoning: {0: 2.9596354166666665, 1: 0.6016410799364743} 
Gamemove: {0: 7.048062015503876, 1: 0.5381792352314431}


In [11]:
X_train_col = train['Input.full_text']

new_deception_test = test["Input.deception_quadrant"].copy()
new_deception_test['Input.deception_quadrant'] = test["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y_test_deception = new_deception_test['Input.deception_quadrant'].tolist()
y_test_rapport = test['Answer.3rapport.yes_label'].tolist()
y_test_share_information = test['Answer.4shareinformation.yes_label'].tolist()
y_test_reasoning = test['Answer.2reasoning.yes_label'].tolist()
y_test_gamemove = test['Answer.1gamemove.yes_label'].tolist()

X_test_col = test['Input.full_text']

In [12]:
le = LabelEncoder()

# def label_preprocessing(y_data, label_encoder):
#     out = label_encoder.fit_transform(y_data).reshape(-1,1)
#     return out

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_train_rapport = le.fit_transform(y_train_rapport)
y_train_rapport = y_train_rapport.reshape(-1,1)

y_train_share_information = le.fit_transform(y_train_share_information)
y_train_share_information = y_train_share_information.reshape(-1,1)

y_train_reasoning = le.fit_transform(y_train_reasoning)
y_train_reasoning = y_train_reasoning.reshape(-1,1)

y_train_gamemove = le.fit_transform(y_train_gamemove)
y_train_gamemove = y_train_gamemove.reshape(-1,1)

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_test_rapport = le.fit_transform(y_test_rapport)
y_test_rapport = y_test_rapport.reshape(-1,1)

y_test_share_information = le.fit_transform(y_test_share_information)
y_test_share_information = y_test_share_information.reshape(-1,1)

y_test_reasoning = le.fit_transform(y_test_reasoning)
y_test_reasoning = y_test_reasoning.reshape(-1,1)

y_test_gamemove = le.fit_transform(y_test_gamemove)
y_test_gamemove = y_test_gamemove.reshape(-1,1)

y_test_deception = le.fit_transform(y_test_deception)
y_test_deception = y_test_deception.reshape(-1,1)

y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

/home/kaiyuan/anaconda3/envs/TF2/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [13]:
max_words = 1000
max_len = 220

tok = Tokenizer(num_words=max_words)

tok.fit_on_texts(X_train_col)
X_train_sequences = tok.texts_to_sequences(X_train_col)
X_train = pad_sequences(X_train_sequences, maxlen=max_len)

X_test_sequences = tok.texts_to_sequences(X_test_col)
X_test = pad_sequences(X_test_sequences, maxlen=max_len)

### Construct individual  models

In [14]:
# Rapport model
rapport_model = models_nn.create_nn_model()
rapport_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, 
                                                                              models_nn.recall_m, models_nn.precision_m])
rapport_model.fit(X_train,y_train_rapport,
                  batch_size=128,
                  epochs=50,
                  validation_data=(X_test, y_test_rapport), 
#                   callbacks=[models_nn.early_stop],
                  class_weight=rapport_class_weight_dict)

Epoch 1/50
72/72 [==============================] - 2s 25ms/step - loss: 0.6935 - accuracy: 0.5793 - f1_m: 0.6516 - recall_m: 0.6062 - precision_m: 0.8424 - val_loss: 0.7008 - val_accuracy: 0.2841 - val_f1_m: 0.3314 - val_recall_m: 0.2059 - val_precision_m: 0.8800
Epoch 2/50
72/72 [==============================] - 1s 20ms/step - loss: 0.6898 - accuracy: 0.5368 - f1_m: 0.6016 - recall_m: 0.5336 - precision_m: 0.8927 - val_loss: 0.6169 - val_accuracy: 0.8637 - val_f1_m: 0.9262 - val_recall_m: 0.9915 - val_precision_m: 0.8695
Epoch 3/50
72/72 [==============================] - 1s 21ms/step - loss: 0.6751 - accuracy: 0.6491 - f1_m: 0.7545 - recall_m: 0.6790 - precision_m: 0.8959 - val_loss: 0.8069 - val_accuracy: 0.3201 - val_f1_m: 0.4005 - val_recall_m: 0.2632 - val_precision_m: 0.8588
Epoch 4/50
72/72 [==============================] - 1s 19ms/step - loss: 0.6535 - accuracy: 0.6253 - f1_m: 0.7358 - recall_m: 0.6258 - precision_m: 0.9146 - val_loss: 0.9307 - val_accuracy: 0.2792 - val_f1

Epoch 32/50
72/72 [==============================] - 1s 20ms/step - loss: 0.3249 - accuracy: 0.8139 - f1_m: 0.8789 - recall_m: 0.7953 - precision_m: 0.9858 - val_loss: 1.0941 - val_accuracy: 0.5989 - val_f1_m: 0.7313 - val_recall_m: 0.6321 - val_precision_m: 0.8696
Epoch 33/50
72/72 [==============================] - 1s 20ms/step - loss: 0.3105 - accuracy: 0.8085 - f1_m: 0.8746 - recall_m: 0.7916 - precision_m: 0.9835 - val_loss: 1.6001 - val_accuracy: 0.4442 - val_f1_m: 0.5678 - val_recall_m: 0.4207 - val_precision_m: 0.8768
Epoch 34/50
72/72 [==============================] - 1s 19ms/step - loss: 0.2962 - accuracy: 0.8113 - f1_m: 0.8699 - recall_m: 0.7837 - precision_m: 0.9873 - val_loss: 0.8954 - val_accuracy: 0.6394 - val_f1_m: 0.7661 - val_recall_m: 0.6829 - val_precision_m: 0.8737
Epoch 35/50
72/72 [==============================] - 1s 19ms/step - loss: 0.3087 - accuracy: 0.8115 - f1_m: 0.8758 - recall_m: 0.7900 - precision_m: 0.9876 - val_loss: 1.4340 - val_accuracy: 0.7005 - va

In [15]:
rapport_pred = rapport_model.predict(X_train)
rapport_pred_test = rapport_model.predict(X_test)

rapport_pred_test_round = rapport_pred_test.round()
precision_recall_fscore_support(y_test_rapport, rapport_pred_test_round, average='macro')

(0.4983727034120735, 0.496795495776429, 0.46940010367328966, None)

In [16]:
# Game move model
gamemove_model = models_nn.create_nn_model()
gamemove_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, 
                                                                               models_nn.recall_m, models_nn.precision_m])
gamemove_model.fit(X_train,
                   y_train_gamemove,
                   batch_size=128,
                   epochs=50,
                   validation_data=(X_test, y_test_gamemove), 
#                    callbacks=[models_nn.early_stop],
                   class_weight=gamemove_class_weight_dict)

Epoch 1/50
72/72 [==============================] - 2s 23ms/step - loss: 0.6931 - accuracy: 0.4845 - f1_m: 0.5574 - recall_m: 0.4896 - precision_m: 0.7589 - val_loss: 0.6519 - val_accuracy: 0.8694 - val_f1_m: 0.9295 - val_recall_m: 0.9205 - val_precision_m: 0.9394
Epoch 2/50
72/72 [==============================] - 1s 19ms/step - loss: 0.6817 - accuracy: 0.6304 - f1_m: 0.7368 - recall_m: 0.6316 - precision_m: 0.9333 - val_loss: 0.6471 - val_accuracy: 0.6328 - val_f1_m: 0.7659 - val_recall_m: 0.6427 - val_precision_m: 0.9499
Epoch 3/50
72/72 [==============================] - 1s 19ms/step - loss: 0.6454 - accuracy: 0.6351 - f1_m: 0.7544 - recall_m: 0.6349 - precision_m: 0.9608 - val_loss: 0.8588 - val_accuracy: 0.3452 - val_f1_m: 0.4782 - val_recall_m: 0.3215 - val_precision_m: 0.9404
Epoch 4/50
72/72 [==============================] - 1s 19ms/step - loss: 0.6037 - accuracy: 0.6411 - f1_m: 0.7605 - recall_m: 0.6354 - precision_m: 0.9669 - val_loss: 0.6069 - val_accuracy: 0.6412 - val_f1

Epoch 32/50
72/72 [==============================] - 1s 19ms/step - loss: 0.2818 - accuracy: 0.8360 - f1_m: 0.8946 - recall_m: 0.8201 - precision_m: 0.9947 - val_loss: 0.8751 - val_accuracy: 0.7661 - val_f1_m: 0.8658 - val_recall_m: 0.8070 - val_precision_m: 0.9350
Epoch 33/50
72/72 [==============================] - 1s 19ms/step - loss: 0.2667 - accuracy: 0.8378 - f1_m: 0.9038 - recall_m: 0.8316 - precision_m: 0.9954 - val_loss: 0.9184 - val_accuracy: 0.7234 - val_f1_m: 0.8370 - val_recall_m: 0.7589 - val_precision_m: 0.9351
Epoch 34/50
72/72 [==============================] - 1s 21ms/step - loss: 0.2883 - accuracy: 0.8355 - f1_m: 0.8991 - recall_m: 0.8288 - precision_m: 0.9962 - val_loss: 1.1283 - val_accuracy: 0.7045 - val_f1_m: 0.8229 - val_recall_m: 0.7331 - val_precision_m: 0.9393
Epoch 35/50
72/72 [==============================] - 1s 19ms/step - loss: 0.3583 - accuracy: 0.8299 - f1_m: 0.8990 - recall_m: 0.8233 - precision_m: 0.9955 - val_loss: 1.0417 - val_accuracy: 0.7467 - va

In [17]:
gamemove_pred = gamemove_model.predict(X_train)
gamemove_pred_test = gamemove_model.predict(X_test)

gamemove_pred_test_round = gamemove_pred_test.round()
precision_recall_fscore_support(y_test_gamemove, gamemove_pred_test_round, average='macro')

(0.5043684722187601, 0.5185239013820258, 0.4039492078285182, None)

In [18]:
# Reasoning model
reasoning_model = models_nn.create_nn_model()
reasoning_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, models_nn.recall_m, models_nn.precision_m])
reasoning_model.fit(X_train,
                    y_train_reasoning,
                    batch_size=128,
                    epochs=50,
                    validation_data=(X_test, y_test_reasoning), 
#                     callbacks=[models_nn.early_stop],
                    class_weight=reasoning_class_weight_dict)

Epoch 1/50
72/72 [==============================] - 2s 23ms/step - loss: 0.6932 - accuracy: 0.4823 - f1_m: 0.5307 - recall_m: 0.4810 - precision_m: 0.8017 - val_loss: 0.6592 - val_accuracy: 0.8219 - val_f1_m: 0.9011 - val_recall_m: 1.0000 - val_precision_m: 0.8212
Epoch 2/50
72/72 [==============================] - 1s 20ms/step - loss: 0.6890 - accuracy: 0.6300 - f1_m: 0.7374 - recall_m: 0.6810 - precision_m: 0.8534 - val_loss: 0.6138 - val_accuracy: 0.7911 - val_f1_m: 0.8804 - val_recall_m: 0.9431 - val_precision_m: 0.8269
Epoch 3/50
72/72 [==============================] - 1s 18ms/step - loss: 0.6690 - accuracy: 0.6197 - f1_m: 0.7245 - recall_m: 0.6325 - precision_m: 0.8780 - val_loss: 0.5422 - val_accuracy: 0.7880 - val_f1_m: 0.8783 - val_recall_m: 0.9414 - val_precision_m: 0.8246
Epoch 4/50
72/72 [==============================] - 1s 19ms/step - loss: 0.6484 - accuracy: 0.6374 - f1_m: 0.7406 - recall_m: 0.6425 - precision_m: 0.8925 - val_loss: 0.5689 - val_accuracy: 0.7379 - val_f1

Epoch 32/50
72/72 [==============================] - 1s 20ms/step - loss: 0.3332 - accuracy: 0.8013 - f1_m: 0.8670 - recall_m: 0.7787 - precision_m: 0.9827 - val_loss: 1.8266 - val_accuracy: 0.5040 - val_f1_m: 0.6206 - val_recall_m: 0.4974 - val_precision_m: 0.8296
Epoch 33/50
72/72 [==============================] - 1s 19ms/step - loss: 0.3016 - accuracy: 0.8113 - f1_m: 0.8714 - recall_m: 0.7845 - precision_m: 0.9840 - val_loss: 1.7774 - val_accuracy: 0.4688 - val_f1_m: 0.5774 - val_recall_m: 0.4457 - val_precision_m: 0.8269
Epoch 34/50
72/72 [==============================] - 1s 19ms/step - loss: 0.3100 - accuracy: 0.8047 - f1_m: 0.8623 - recall_m: 0.7708 - precision_m: 0.9853 - val_loss: 1.5478 - val_accuracy: 0.5836 - val_f1_m: 0.7098 - val_recall_m: 0.6245 - val_precision_m: 0.8241
Epoch 35/50
72/72 [==============================] - 2s 21ms/step - loss: 0.2989 - accuracy: 0.8103 - f1_m: 0.8658 - recall_m: 0.7788 - precision_m: 0.9838 - val_loss: 1.8933 - val_accuracy: 0.5822 - va

In [19]:
reasoning_pred = reasoning_model.predict(X_train)
reasoning_pred_test = reasoning_model.predict(X_test)

reasoning_pred_test_round = reasoning_pred_test.round()
precision_recall_fscore_support(y_test_reasoning, reasoning_pred_test_round, average='macro')

(0.5209828305968184, 0.5277827319025821, 0.5184201977987054, None)

In [20]:
# Share Information model
shareinfo_model = models_nn.create_nn_model()
shareinfo_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, models_nn.recall_m, models_nn.precision_m])
shareinfo_model.fit(X_train,
                    y_train_share_information,
                    batch_size=128,
                    epochs=50,
                    validation_data=(X_test, y_test_share_information), 
#                     callbacks=[models_nn.early_stop],
                    class_weight=share_info_class_weight_dict)

Epoch 1/50
72/72 [==============================] - 2s 25ms/step - loss: 0.6934 - accuracy: 0.6098 - f1_m: 0.6832 - recall_m: 0.6555 - precision_m: 0.7944 - val_loss: 0.7038 - val_accuracy: 0.2203 - val_f1_m: 0.1610 - val_recall_m: 0.0890 - val_precision_m: 0.8800
Epoch 2/50
72/72 [==============================] - 1s 18ms/step - loss: 0.6876 - accuracy: 0.5875 - f1_m: 0.6929 - recall_m: 0.6047 - precision_m: 0.8776 - val_loss: 0.7076 - val_accuracy: 0.4622 - val_f1_m: 0.5730 - val_recall_m: 0.4301 - val_precision_m: 0.8618
Epoch 3/50
72/72 [==============================] - 2s 21ms/step - loss: 0.6642 - accuracy: 0.5727 - f1_m: 0.6776 - recall_m: 0.5602 - precision_m: 0.8929 - val_loss: 0.7158 - val_accuracy: 0.4741 - val_f1_m: 0.5893 - val_recall_m: 0.4492 - val_precision_m: 0.8605
Epoch 4/50
72/72 [==============================] - 1s 19ms/step - loss: 0.6387 - accuracy: 0.5965 - f1_m: 0.6986 - recall_m: 0.5751 - precision_m: 0.9135 - val_loss: 0.5209 - val_accuracy: 0.7792 - val_f1

Epoch 32/50
72/72 [==============================] - 1s 19ms/step - loss: 0.3559 - accuracy: 0.7738 - f1_m: 0.8467 - recall_m: 0.7474 - precision_m: 0.9835 - val_loss: 1.2843 - val_accuracy: 0.5822 - val_f1_m: 0.7087 - val_recall_m: 0.6052 - val_precision_m: 0.8586
Epoch 33/50
72/72 [==============================] - 1s 20ms/step - loss: 0.3417 - accuracy: 0.7762 - f1_m: 0.8474 - recall_m: 0.7489 - precision_m: 0.9810 - val_loss: 0.9301 - val_accuracy: 0.6126 - val_f1_m: 0.7397 - val_recall_m: 0.6561 - val_precision_m: 0.8497
Epoch 34/50
72/72 [==============================] - 1s 19ms/step - loss: 0.3312 - accuracy: 0.7853 - f1_m: 0.8499 - recall_m: 0.7513 - precision_m: 0.9835 - val_loss: 1.1768 - val_accuracy: 0.5919 - val_f1_m: 0.7204 - val_recall_m: 0.6254 - val_precision_m: 0.8528
Epoch 35/50
72/72 [==============================] - 1s 19ms/step - loss: 0.3326 - accuracy: 0.7826 - f1_m: 0.8541 - recall_m: 0.7594 - precision_m: 0.9806 - val_loss: 1.6834 - val_accuracy: 0.5594 - va

In [21]:
shareinfo_pred = shareinfo_model.predict(X_train)
shareinfo_pred_test = shareinfo_model.predict(X_test)

shareinfo_pred_test_round = shareinfo_pred_test.round()
precision_recall_fscore_support(y_test_share_information, shareinfo_pred_test_round, average='macro')

(0.5084688346883469, 0.5146760240112994, 0.48500380349430694, None)

In [22]:
# Deception model
deception_model = models_nn.create_nn_model()
deception_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, models_nn.recall_m, models_nn.precision_m])
deception_model.fit(X_train,
                    y_train_deception,
                    epochs=50,
                    validation_data=(X_test, y_test_deception), 
#                     callbacks=[models_nn.early_stop],
                    class_weight=deception_class_weight_dict)

Epoch 1/50
285/285 [==============================] - 4s 14ms/step - loss: 0.7013 - accuracy: 0.8251 - f1_m: 0.8855 - recall_m: 0.8608 - precision_m: 0.9511 - val_loss: 0.6738 - val_accuracy: 0.8193 - val_f1_m: 0.8997 - val_recall_m: 0.8517 - val_precision_m: 0.9566
Epoch 2/50
285/285 [==============================] - 4s 14ms/step - loss: 0.6836 - accuracy: 0.7077 - f1_m: 0.7933 - recall_m: 0.7238 - precision_m: 0.9609 - val_loss: 0.7021 - val_accuracy: 0.4204 - val_f1_m: 0.5704 - val_recall_m: 0.4145 - val_precision_m: 0.9633
Epoch 3/50
285/285 [==============================] - 4s 14ms/step - loss: 0.6655 - accuracy: 0.6009 - f1_m: 0.7146 - recall_m: 0.6009 - precision_m: 0.9665 - val_loss: 0.6307 - val_accuracy: 0.5901 - val_f1_m: 0.7348 - val_recall_m: 0.6033 - val_precision_m: 0.9556
Epoch 4/50
285/285 [==============================] - 4s 14ms/step - loss: 0.6388 - accuracy: 0.6147 - f1_m: 0.7300 - recall_m: 0.6121 - precision_m: 0.9712 - val_loss: 0.5858 - val_accuracy: 0.6486 

Epoch 32/50
285/285 [==============================] - 4s 14ms/step - loss: 0.4085 - accuracy: 0.7994 - f1_m: 0.8777 - recall_m: 0.7954 - precision_m: 0.9918 - val_loss: 0.3361 - val_accuracy: 0.8747 - val_f1_m: 0.9327 - val_recall_m: 0.9149 - val_precision_m: 0.9537
Epoch 33/50
285/285 [==============================] - 4s 13ms/step - loss: 0.4389 - accuracy: 0.7972 - f1_m: 0.8752 - recall_m: 0.7953 - precision_m: 0.9884 - val_loss: 0.5852 - val_accuracy: 0.7370 - val_f1_m: 0.8468 - val_recall_m: 0.7700 - val_precision_m: 0.9466
Epoch 34/50
285/285 [==============================] - 4s 12ms/step - loss: 0.3699 - accuracy: 0.8307 - f1_m: 0.9011 - recall_m: 0.8298 - precision_m: 0.9917 - val_loss: 0.6387 - val_accuracy: 0.6561 - val_f1_m: 0.7890 - val_recall_m: 0.6811 - val_precision_m: 0.9468
Epoch 35/50
285/285 [==============================] - 4s 13ms/step - loss: 0.3596 - accuracy: 0.8323 - f1_m: 0.9015 - recall_m: 0.8310 - precision_m: 0.9912 - val_loss: 0.8097 - val_accuracy: 0.5

In [23]:
deception_pred = deception_model.predict(X_train)
deception_pred_test = deception_model.predict(X_test)
deception_pred_test_round = deception_pred_test.round()
precision_recall_fscore_support(y_test_deception, deception_pred_test_round, average='macro')

(0.49688759439454816, 0.4876478459098718, 0.4567600057518094, None)

In [24]:
# deception_pred_test

In [25]:
# unique, counts = np.unique(deception_pred_test_round, return_counts=True)

In [26]:
# print(unique, counts)

In [27]:
# deception_model.history.history['val_f1_m']

### One hot encodings

In [28]:
# Train encodings
pred_df_arr_full = []
pred_df_arr = []
for i in range(0, len(gamemove_pred)):
    pred_obj_1 = {}
    pred_obj_1['gamemove'] = gamemove_pred[i][0]
    pred_obj_1['reasoning'] = reasoning_pred[i][0]
    pred_obj_1['shareinfo'] = shareinfo_pred[i][0]
    pred_df_arr.append(pred_obj_1)
    
    pred_obj_2 = pred_obj_1.copy()
    pred_obj_2['rapport'] = rapport_pred[i][0]
    pred_df_arr_full.append(pred_obj_2)
    
pred_df_full = pd.DataFrame(pred_df_arr_full)
pred_df = pd.DataFrame(pred_df_arr)

In [29]:
# Test encodings
pred_test_df_arr_full = []
pred_test_df_arr = []

for i in range(0, len(gamemove_pred_test)):
    pred_obj_1 = {}
    pred_obj_1['gamemove'] = gamemove_pred_test[i][0]
    pred_obj_1['reasoning'] = reasoning_pred_test[i][0]
    pred_obj_1['shareinfo'] = shareinfo_pred_test[i][0]
    pred_test_df_arr.append(pred_obj_1)
    
    pred_obj_2 = pred_obj_1.copy()
    pred_obj_2['rapport'] = rapport_pred_test[i][0]
    pred_test_df_arr_full.append(pred_obj_2)
    
pred_test_df_full = pd.DataFrame(pred_test_df_arr_full)
pred_test_df = pd.DataFrame(pred_test_df_arr)

### Joint Model with one hot encoding 

In [30]:
print('Joint full model with one hot encoding, predicting deception')
joint_full_model = models_nn.create_joint_model(pred_df_full)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_test_df_full,y_test_deception), 
#                                callbacks=[models_nn.callback], 
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception
Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 12        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6982 - acc: 0.7387 - f1_m: 0.8367 - precision_m: 0.9505 - recall_m: 0.7639 - val_loss: 0.6698 - val_acc: 0.5048 - val_

143/143 [==============================] - 0s 2ms/step - loss: 0.6927 - acc: 0.9051 - f1_m: 0.9499 - precision_m: 0.9520 - recall_m: 0.9486 - val_loss: 0.6921 - val_acc: 0.8984 - val_f1_m: 0.9462 - val_precision_m: 0.9508 - val_recall_m: 0.9426
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6928 - acc: 0.9049 - f1_m: 0.9488 - precision_m: 0.9517 - recall_m: 0.9471 - val_loss: 0.6931 - val_acc: 0.8993 - val_f1_m: 0.9467 - val_precision_m: 0.9509 - val_recall_m: 0.9434
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6927 - acc: 0.8037 - f1_m: 0.8469 - precision_m: 0.9463 - recall_m: 0.8390 - val_loss: 0.6921 - val_acc: 0.8997 - val_f1_m: 0.9469 - val_precision_m: 0.9509 - val_recall_m: 0.9439
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6927 - acc: 0.8822 - f1_m: 0.9292 - precision_m: 0.9522 - recall_m: 0.9245 - val_loss: 0.6907 - val_acc: 0.9081 - val_f1_m: 0.9515 - val_precision_m: 0.9508 - val_r

In [31]:
joint_predict = joint_full_model.predict(pred_test_df_full)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.503042566264805, 0.5029651777454738, 0.5029986395013109, None)

In [32]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 109 2165]


In [33]:
print('Joint full model with one hot encoding, predicting rapport')
joint_full_model = models_nn.create_joint_model(pred_df)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_test_df,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport
Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 8         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 12        
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6766 - acc: 0.7850 - f1_m: 0.8710 - precision_m: 0.8995 - recall_m: 0.8475 - val_loss: 0.6454 - val_acc: 0.6887 - val_f1

143/143 [==============================] - 0s 2ms/step - loss: 0.5938 - acc: 0.7095 - f1_m: 0.8076 - precision_m: 0.9313 - recall_m: 0.7153 - val_loss: 0.7235 - val_acc: 0.5673 - val_f1_m: 0.7038 - val_precision_m: 0.8671 - val_recall_m: 0.5957
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.5939 - acc: 0.7178 - f1_m: 0.8133 - precision_m: 0.9313 - recall_m: 0.7248 - val_loss: 0.7586 - val_acc: 0.5466 - val_f1_m: 0.6836 - val_precision_m: 0.8675 - val_recall_m: 0.5674
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.5939 - acc: 0.7110 - f1_m: 0.8096 - precision_m: 0.9320 - recall_m: 0.7178 - val_loss: 0.7396 - val_acc: 0.5563 - val_f1_m: 0.6930 - val_precision_m: 0.8673 - val_recall_m: 0.5803
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.5939 - acc: 0.7114 - f1_m: 0.8085 - precision_m: 0.9316 - recall_m: 0.7172 - val_loss: 0.7376 - val_acc: 0.5572 - val_f1_m: 0.6938 - val_precision_m: 0.8675 - val_r

In [34]:
joint_predict = joint_full_model.predict(pred_test_df)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.4966906202723147, 0.4928243225316272, 0.44314515099453217, None)

In [35]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 952 1322]


## Weighted against Throughput, WorkTime, PC Agreement & Text Length

In [36]:
# Train weighted encodings
pred_df_full_throughput, pred_df_throughput, pred_df_full_worktime, pred_df_worktime, pred_df_full_agreement, pred_df_agreement, pred_df_full_textlength, pred_df_textlength, pred_df_full_special, pred_df_special = metadata_options.construct_weighted_dataframe(indices_train, df_throughput, df_worktime, df_agreement, df_textlength, df_special, pred_df, pred_df_full)

In [37]:
# Test weighted encodings
pred_df_full_throughput_test, pred_df_throughput_test, pred_df_full_worktime_test, pred_df_worktime_test, pred_df_full_agreement_test, pred_df_agreement_test, pred_df_full_textlength_test, pred_df_textlength_test, pred_df_full_special_test, pred_df_special_test = metadata_options.construct_weighted_dataframe(indices_test, df_throughput, df_worktime, df_agreement, df_textlength, df_special, pred_test_df, pred_test_df_full)

## Throughput only

### Joint Model with one hot encoding 

In [38]:
print('Joint full model with one hot encoding, predicting deception, weighted by throughput')
joint_full_model = models_nn.create_joint_model(pred_df_full_throughput)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_throughput, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_throughput_test,y_test_deception), 
#                                callbacks=[models_nn.callback], 
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by throughput
Model: "functional_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_9 (Dense)              (None, 4)                 12        
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6931 - acc: 0.3061 - f1_m: 0.3289 - precision_m: 0.3998 - recall_m: 0.2876 - val_loss: 0.6948 

143/143 [==============================] - 0s 2ms/step - loss: 0.6923 - acc: 0.7356 - f1_m: 0.8424 - precision_m: 0.9506 - recall_m: 0.7590 - val_loss: 0.6940 - val_acc: 0.6746 - val_f1_m: 0.7996 - val_precision_m: 0.9510 - val_recall_m: 0.6918
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6923 - acc: 0.6291 - f1_m: 0.7628 - precision_m: 0.9518 - recall_m: 0.6398 - val_loss: 0.6982 - val_acc: 0.6284 - val_f1_m: 0.7642 - val_precision_m: 0.9520 - val_recall_m: 0.6404
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6922 - acc: 0.6016 - f1_m: 0.7446 - precision_m: 0.9530 - recall_m: 0.6142 - val_loss: 0.6958 - val_acc: 0.6596 - val_f1_m: 0.7885 - val_precision_m: 0.9517 - val_recall_m: 0.6750
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6923 - acc: 0.6289 - f1_m: 0.7649 - precision_m: 0.9529 - recall_m: 0.6440 - val_loss: 0.6923 - val_acc: 0.6878 - val_f1_m: 0.8090 - val_precision_m: 0.9513 - val_r

In [39]:
joint_predict = joint_full_model.predict(pred_df_full_throughput_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.5022250020506931, 0.5098577705827937, 0.45065735750449126, None)

In [40]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 668 1606]


In [41]:
y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

In [42]:
print('Joint full model with one hot encoding, predicting rapport, weighted by throughput')
joint_full_model = models_nn.create_joint_model(pred_df_throughput)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_throughput, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64,
                               validation_data=(pred_df_throughput_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by throughput
Model: "functional_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_12 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6932 - acc: 0.3176 - f1_m: 0.2351 - precision_m: 0.2995 - recall_m: 0.2537 - val_loss: 0.6934 - 

143/143 [==============================] - 0s 2ms/step - loss: 0.6696 - acc: 0.7629 - f1_m: 0.8580 - precision_m: 0.8860 - recall_m: 0.8343 - val_loss: 0.6908 - val_acc: 0.7067 - val_f1_m: 0.8224 - val_precision_m: 0.8641 - val_recall_m: 0.7868
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6694 - acc: 0.7577 - f1_m: 0.8533 - precision_m: 0.8857 - recall_m: 0.8258 - val_loss: 0.6880 - val_acc: 0.7142 - val_f1_m: 0.8278 - val_precision_m: 0.8651 - val_recall_m: 0.7958
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6690 - acc: 0.7680 - f1_m: 0.8613 - precision_m: 0.8860 - recall_m: 0.8398 - val_loss: 0.6895 - val_acc: 0.7080 - val_f1_m: 0.8233 - val_precision_m: 0.8644 - val_recall_m: 0.7884
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6688 - acc: 0.7683 - f1_m: 0.8616 - precision_m: 0.8860 - recall_m: 0.8405 - val_loss: 0.6882 - val_acc: 0.7120 - val_f1_m: 0.8261 - val_precision_m: 0.8652 - val_r

In [43]:
joint_predict = joint_full_model.predict(pred_df_throughput_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.48810606782244237, 0.4827904340380371, 0.4809823864976739, None)

In [44]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 464 1810]


## WorkTime only

### Joint Model with one hot encoding 

In [45]:
print('Joint full model with one hot encoding, predicting deception, weighted by worktime')
joint_full_model = models_nn.create_joint_model(pred_df_full_worktime)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_worktime, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_worktime_test,y_test_deception), 
#                                callbacks=[models_nn.callback],
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by worktime
Model: "functional_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_15 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6934 - acc: 0.2991 - f1_m: 0.3262 - precision_m: 0.4385 - recall_m: 0.2818 - val_loss: 0.6929 - 

Epoch 29/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6926 - acc: 0.9039 - f1_m: 0.9493 - precision_m: 0.9512 - recall_m: 0.9482 - val_loss: 0.6907 - val_acc: 0.8980 - val_f1_m: 0.9459 - val_precision_m: 0.9495 - val_recall_m: 0.9431
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6927 - acc: 0.9118 - f1_m: 0.9528 - precision_m: 0.9515 - recall_m: 0.9553 - val_loss: 0.6927 - val_acc: 0.8870 - val_f1_m: 0.9398 - val_precision_m: 0.9494 - val_recall_m: 0.9312
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6927 - acc: 0.8729 - f1_m: 0.9251 - precision_m: 0.9507 - recall_m: 0.9138 - val_loss: 0.6945 - val_acc: 0.5950 - val_f1_m: 0.7374 - val_precision_m: 0.9522 - val_recall_m: 0.6038
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6927 - acc: 0.4248 - f1_m: 0.4880 - precision_m: 0.7790 - recall_m: 0.4220 - val_loss: 0.6933 - val_acc: 0.8918 - val_f1_m: 0.9425 - val_precision_m: 0.

In [46]:
joint_predict = joint_full_model.predict(pred_df_full_worktime_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.49224374021909234, 0.49017526760935637, 0.49087328168685923, None)

In [47]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 144 2130]


In [48]:
print('Joint full model with one hot encoding, predicting rapport, weighted by worktime')
joint_full_model = models_nn.create_joint_model(pred_df_worktime)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_worktime, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64,
                               validation_data=(pred_df_worktime_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by worktime
Model: "functional_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_18 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6887 - acc: 0.4074 - f1_m: 0.4840 - precision_m: 0.8334 - recall_m: 0.3513 - val_loss: 0.7242 - va

143/143 [==============================] - 0s 2ms/step - loss: 0.6480 - acc: 0.6421 - f1_m: 0.7533 - precision_m: 0.9150 - recall_m: 0.6440 - val_loss: 0.6889 - val_acc: 0.5787 - val_f1_m: 0.7070 - val_precision_m: 0.8859 - val_recall_m: 0.5909
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6474 - acc: 0.6413 - f1_m: 0.7543 - precision_m: 0.9151 - recall_m: 0.6442 - val_loss: 0.6983 - val_acc: 0.5708 - val_f1_m: 0.6980 - val_precision_m: 0.8884 - val_recall_m: 0.5774
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6476 - acc: 0.6454 - f1_m: 0.7586 - precision_m: 0.9127 - recall_m: 0.6527 - val_loss: 0.7102 - val_acc: 0.5580 - val_f1_m: 0.6856 - val_precision_m: 0.8882 - val_recall_m: 0.5609
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6466 - acc: 0.6326 - f1_m: 0.7453 - precision_m: 0.9142 - recall_m: 0.6345 - val_loss: 0.6882 - val_acc: 0.5800 - val_f1_m: 0.7085 - val_precision_m: 0.8849 - val_r

In [49]:
joint_predict = joint_full_model.predict(pred_df_worktime_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.5182004444840673, 0.5393930048773933, 0.4703051403575399, None)

In [50]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 946 1328]


## PC Agreement only

### Joint Model with one hot encoding 

In [51]:
print('Joint full model with one hot encoding, predicting deception, weighted by PC Agreement')
joint_full_model = models_nn.create_joint_model(pred_df_full_agreement)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_agreement, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_agreement_test,y_test_deception), 
#                                callbacks=[models_nn.callback],
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by PC Agreement
Model: "functional_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_21 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 4ms/step - loss: 0.6935 - acc: 0.9309 - f1_m: 0.9615 - precision_m: 0.9510 - recall_m: 0.9751 - val_loss: 0.688

Epoch 29/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6931 - acc: 0.1623 - f1_m: 0.2083 - precision_m: 0.8782 - recall_m: 0.1280 - val_loss: 0.6982 - val_acc: 0.3725 - val_f1_m: 0.5164 - val_precision_m: 0.9567 - val_recall_m: 0.3556
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6930 - acc: 0.3701 - f1_m: 0.5097 - precision_m: 0.9527 - recall_m: 0.3562 - val_loss: 0.6940 - val_acc: 0.5321 - val_f1_m: 0.6811 - val_precision_m: 0.9508 - val_recall_m: 0.5330
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6929 - acc: 0.5207 - f1_m: 0.6704 - precision_m: 0.9541 - recall_m: 0.5216 - val_loss: 0.6918 - val_acc: 0.6113 - val_f1_m: 0.7493 - val_precision_m: 0.9526 - val_recall_m: 0.6199
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6929 - acc: 0.5510 - f1_m: 0.6988 - precision_m: 0.9526 - recall_m: 0.5575 - val_loss: 0.6922 - val_acc: 0.5897 - val_f1_m: 0.7314 - val_precision_m: 0.

In [52]:
joint_predict = joint_full_model.predict(pred_df_full_throughput_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.5174636277561528, 0.5388951007265266, 0.4395372446221365, None)

In [53]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [1142 1132]


In [54]:
y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

In [55]:
print('Joint full model with one hot encoding, predicting rapport, weighted by PC Agreement')
joint_full_model = models_nn.create_joint_model(pred_df_agreement)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_agreement, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_agreement_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by PC Agreement
Model: "functional_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_23 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_24 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6828 - acc: 0.8417 - f1_m: 0.9099 - precision_m: 0.8801 - recall_m: 0.9449 - val_loss: 0.6498 

143/143 [==============================] - 0s 2ms/step - loss: 0.6167 - acc: 0.6451 - f1_m: 0.7516 - precision_m: 0.9222 - recall_m: 0.6377 - val_loss: 0.6929 - val_acc: 0.5215 - val_f1_m: 0.6561 - val_precision_m: 0.8740 - val_recall_m: 0.5291
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6166 - acc: 0.6326 - f1_m: 0.7431 - precision_m: 0.9279 - recall_m: 0.6229 - val_loss: 0.6908 - val_acc: 0.5242 - val_f1_m: 0.6590 - val_precision_m: 0.8733 - val_recall_m: 0.5332
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6158 - acc: 0.6477 - f1_m: 0.7556 - precision_m: 0.9240 - recall_m: 0.6424 - val_loss: 0.6890 - val_acc: 0.5273 - val_f1_m: 0.6621 - val_precision_m: 0.8732 - val_recall_m: 0.5372
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6154 - acc: 0.6455 - f1_m: 0.7549 - precision_m: 0.9267 - recall_m: 0.6396 - val_loss: 0.6888 - val_acc: 0.5299 - val_f1_m: 0.6645 - val_precision_m: 0.8739 - val_r

In [56]:
joint_predict = joint_full_model.predict(pred_df_throughput_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.4630042545372307, 0.4278504151211251, 0.42664104979465906, None)

In [57]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 736 1538]


## Text Length only

### Joint Model with one hot encoding 

In [58]:
print('Joint full model with one hot encoding, predicting deception, weighted by text length')
joint_full_model = models_nn.create_joint_model(pred_df_full_textlength)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_textlength, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_textlength_test,y_test_deception), 
#                                callbacks=[models_nn.callback],
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by text length
Model: "functional_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_26 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_27 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6938 - acc: 0.9450 - f1_m: 0.9695 - precision_m: 0.9514 - recall_m: 0.9939 - val_loss: 0.6902

143/143 [==============================] - 0s 2ms/step - loss: 0.6930 - acc: 0.9264 - f1_m: 0.9617 - precision_m: 0.9520 - recall_m: 0.9724 - val_loss: 0.6924 - val_acc: 0.9164 - val_f1_m: 0.9559 - val_precision_m: 0.9517 - val_recall_m: 0.9608
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6930 - acc: 0.6607 - f1_m: 0.6800 - precision_m: 0.6981 - recall_m: 0.6833 - val_loss: 0.6916 - val_acc: 0.8769 - val_f1_m: 0.9339 - val_precision_m: 0.9509 - val_recall_m: 0.9183
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6929 - acc: 0.9020 - f1_m: 0.9481 - precision_m: 0.9512 - recall_m: 0.9461 - val_loss: 0.6915 - val_acc: 0.8967 - val_f1_m: 0.9450 - val_precision_m: 0.9507 - val_recall_m: 0.9402
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6929 - acc: 0.9136 - f1_m: 0.9546 - precision_m: 0.9514 - recall_m: 0.9588 - val_loss: 0.6908 - val_acc: 0.9160 - val_f1_m: 0.9556 - val_precision_m: 0.9520 - val_r

In [59]:
joint_predict = joint_full_model.predict(pred_df_full_textlength_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.5182411052873119, 0.5155940266948593, 0.5166478037185047, None)

In [60]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [  95 2179]


In [61]:
print('Joint full model with one hot encoding, predicting rapport, weighted by text length')
joint_full_model = models_nn.create_joint_model(pred_df_textlength)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_textlength, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_textlength_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by text length
Model: "functional_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 3)]               0         
_________________________________________________________________
dense_29 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_30 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6932 - acc: 0.1396 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_lo

143/143 [==============================] - 0s 2ms/step - loss: 0.6932 - acc: 0.7546 - f1_m: 0.7944 - precision_m: 0.7389 - recall_m: 0.8601 - val_loss: 0.6920 - val_acc: 0.8712 - val_f1_m: 0.9299 - val_precision_m: 0.8698 - val_recall_m: 1.0000
Epoch 29/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6932 - acc: 0.8604 - f1_m: 0.9210 - precision_m: 0.8564 - recall_m: 1.0000 - val_loss: 0.6913 - val_acc: 0.8712 - val_f1_m: 0.9299 - val_precision_m: 0.8698 - val_recall_m: 1.0000
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6932 - acc: 0.1957 - f1_m: 0.0762 - precision_m: 0.0698 - recall_m: 0.0839 - val_loss: 0.6936 - val_acc: 0.1288 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6932 - acc: 0.5549 - f1_m: 0.5417 - precision_m: 0.5033 - recall_m: 0.5874 - val_loss: 0.6929 - val_acc: 0.8712 - val_f1_m: 0.9299 - val_precision_m: 0.

In [62]:
joint_predict = joint_full_model.predict(pred_df_textlength_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

/home/kaiyuan/anaconda3/envs/TF2/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.06442392260334213, 0.5, 0.11414102064666927, None)

In [63]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0.] [2274]


## Other special options (either SP1, SP2, SP3, RAND_UNI, or RAND_NORM)

### Joint Model with one hot encoding 

In [64]:
print('Joint full model with one hot encoding, predicting deception, weighted by special option')
joint_full_model = models_nn.create_joint_model(pred_df_full_special)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_special, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_special_test,y_test_deception), 
#                                callbacks=[models_nn.callback],
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by special option
Model: "functional_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 4)]               0         
_________________________________________________________________
dense_32 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_33 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_34 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6931 - acc: 0.1303 - f1_m: 0.1464 - precision_m: 0.4227 - recall_m: 0.0957 - val_loss: 0.6

143/143 [==============================] - 0s 2ms/step - loss: 0.6928 - acc: 0.7157 - f1_m: 0.8270 - precision_m: 0.9524 - recall_m: 0.7383 - val_loss: 0.6906 - val_acc: 0.6759 - val_f1_m: 0.8013 - val_precision_m: 0.9482 - val_recall_m: 0.6963
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6928 - acc: 0.6284 - f1_m: 0.7635 - precision_m: 0.9533 - recall_m: 0.6404 - val_loss: 0.6874 - val_acc: 0.6917 - val_f1_m: 0.8136 - val_precision_m: 0.9484 - val_recall_m: 0.7146
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6927 - acc: 0.6752 - f1_m: 0.7991 - precision_m: 0.9519 - recall_m: 0.6943 - val_loss: 0.6841 - val_acc: 0.7230 - val_f1_m: 0.8363 - val_precision_m: 0.9483 - val_recall_m: 0.7500
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6929 - acc: 0.7133 - f1_m: 0.8275 - precision_m: 0.9527 - recall_m: 0.7357 - val_loss: 0.6930 - val_acc: 0.6262 - val_f1_m: 0.7627 - val_precision_m: 0.9482 - val_r

In [65]:
joint_predict = joint_full_model.predict(pred_df_full_special_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.4953964785973823, 0.47748034227567066, 0.4208990460668233, None)

In [66]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 808 1466]


In [67]:
y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

In [68]:
print('Joint full model with one hot encoding, predicting rapport, weighted by special option')
joint_full_model = models_nn.create_joint_model(pred_df_special)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_special, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64,
                               validation_data=(pred_df_special_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by special option
Model: "functional_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 3)]               0         
_________________________________________________________________
dense_35 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_36 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_37 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6936 - acc: 0.6796 - f1_m: 0.7908 - precision_m: 0.8865 - recall_m: 0.7253 - val_loss: 0.683

143/143 [==============================] - 0s 2ms/step - loss: 0.6483 - acc: 0.7399 - f1_m: 0.8357 - precision_m: 0.9089 - recall_m: 0.7759 - val_loss: 0.6758 - val_acc: 0.7040 - val_f1_m: 0.8145 - val_precision_m: 0.8883 - val_recall_m: 0.7539
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6468 - acc: 0.7414 - f1_m: 0.8369 - precision_m: 0.9087 - recall_m: 0.7782 - val_loss: 0.6729 - val_acc: 0.7071 - val_f1_m: 0.8167 - val_precision_m: 0.8887 - val_recall_m: 0.7573
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6456 - acc: 0.7446 - f1_m: 0.8401 - precision_m: 0.9099 - recall_m: 0.7831 - val_loss: 0.6676 - val_acc: 0.7067 - val_f1_m: 0.8168 - val_precision_m: 0.8878 - val_recall_m: 0.7583
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6446 - acc: 0.7445 - f1_m: 0.8363 - precision_m: 0.9058 - recall_m: 0.7796 - val_loss: 0.6826 - val_acc: 0.6873 - val_f1_m: 0.8017 - val_precision_m: 0.8863 - val_r

In [69]:
joint_predict = joint_full_model.predict(pred_df_special_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.5302778011972009, 0.5544472833212446, 0.5198666518977249, None)

In [70]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 638 1636]
